# Programming a neural network layer

[Keras](https://keras.io) is a high-level deep-learning framework building on top of [TensorFlow](https://www.tensorflow.org). These frameworks follow the _symbol-to-symbol derivatives_ approach, i.e. automatically derive a computational graph to calculate derivatives. You just need to declare your inputs as TensorFlow variables and use TensorFlow operations on them to compute the forward pass.  

## Task 6.1

Work through the [Keras tutorial on custom layers](https://keras.io/guides/making_new_layers_and_models_via_subclassing) to learn how to create your own neural network layer.  
Create a custom Keras layer that computes Gaussian basis functions, i.e. a layer that maps an input vector $\mathbf x \in \mathbb R^n$ onto an output vector $\mathbf y = f(\mathbf x) \in \mathbb R^m$ as follows:
\begin{align}
  f: \mathbf x \in \mathbb R^n \mapsto \left[w_i \exp\left(-\frac{\|\mathbf x - \boldsymbol\mu_i\|^2}{\sigma_i^2}\right)\right]_{i=1..m} \in \mathbb R^m
\end{align}

Instead of projecting an input $\mathbf x$ onto a weight vector $\mathbf w$ as the standard neuron function $f(\mathbf x) = \sigma(\mathbf w \cdot \mathbf x + b)$ does, the Gaussian basis function becomes active (with weight $w_i$) for all inputs $\mathbf x$ close to a prototype $\boldsymbol \mu_i$. This activation quickly decays with increasing distance of $\mathbf x$ to $\boldsymbol \mu_i$. The parameter $\sigma_i$ controls the width of the Gaussian, i.e. the size of the active region.

For efficient tensor-based operations you need to correctly _broadcast_ the tensors for the difference operation: TensorFlow will pass an input matrix of shape `(batch size, input dim)` for $\mathbf X$, while you will have a matrix of centers $\boldsymbol \mu$ of shape `(input dim, #units)`. To correctly [broadcast](https://numpy.org/doc/stable/user/basics.broadcasting.html) them together, you will need Keras' [`expand_dims()`](https://www.tensorflow.org/api_docs/python/tf/keras/backend/expand_dims) function to extend $\mathbf X$'s shape to `(batch size, input dim, 1)`:

In [1]:
import numpy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K

class RBFLayer(keras.layers.Layer):
    def __init__(self,units,gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units=units
        self.gamma=gamma

    def build(self, input_shape):
        #super(RBFLayer, self).build(input_shape)
        self.mu=self.add_weight(name='mu',shape=(int(input_shape[1]),self.units),
                                initializer='uniform',
                                trainable=True)
        self.w=self.add_weight(name='w',
                               shape=(self.units,),
                               initializer='uniform',
                               trainable=True)

    def call(self, X):
        diff=K.expand_dims(X)-self.mu
        l2=K.sum(K.pow(diff,2),axis=1)
        res=K.exp(-1*self.gamma *l2)
        X=res*self.w
        return X


class InitCentersRandom(keras.initializers.Initializer):
    """ Initializer to initialize centers of RBF network from random samples of the data set."""

    def __init__(self, X):
        self.X = X

    def __call__(self, shape, dtype=None):
        idx=np.random.randint(self.X.shape[0],size=shape[0])
        #pass
        return self.X[idx,:]



X = tf.ones((3, 5))  # input tensor X with batch dimension 3 and data dim N=5
#mu = tf.ones((5, 2))  # tensor mu with data dim N=5 and 2 units
#diffs = K.expand_dims(X) - mu  # diffs tensor: 3 x 5 x 2
print(X.shape)
print(X)
layer=RBFLayer(2,gamma=1)
Y=layer(X)
print(Y.shape)
print(Y)


(3, 5)
tf.Tensor(
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]], shape=(3, 5), dtype=float32)
(3, 2)
tf.Tensor(
[[-6.6189721e-05  2.0400532e-04]
 [-6.6189721e-05  2.0400532e-04]
 [-6.6189721e-05  2.0400532e-04]], shape=(3, 2), dtype=float32)


The output of above code is the shape of resulting tensor when applying the RBFLayer to input tensor 'X'. Given that we are instantiating the 'RBFLayer' with 2 units and our input tensor 'X' has a batch size of 3, the output is in the shape of resulting tensor, which should be '(3,2)'. This also means the layer is processing the 3 input samples through 2 neurons int he RBFLayer, so we are getting 2 outputs for each inoput sample.

## Task 6.2

Compare the performance of such a Gaussian basis function layer with that of a standard [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) layer on the MNIST dataset.  
Hint: Utilize existing tutorials on setting up your first MNIST MLP with Keras, e.g. https://www.tensorflow.org/guide/keras/train_and_evaluate.

To achieve decent performance, you want to:
- Initialize the centers $\boldsymbol \mu_i$ from random data samples $\mathbf x$ (create a custom [initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/Initializer))
- Initialize $\sigma_i$ to the typical in-class distance between data points.  
  Use [`scipy.spatial.distance_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance_matrix.html) to compute this statistics on a random selection of your input data.  
  (Doing it on the full dataset will probably exhaust your memory.)
- Initialize $w_i = 1$

Questions:
- How many parameters each of those networks have?
- Which network trains faster / easier?

In [2]:
#Let's define functions to create model with 'Dense' layers and 'RBFLayer'
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

def create_dense_model(input_shape,num_classes):
  model=Sequential()
  model.add(Flatten(input_shape=input_shape))
  model.add(Dense(64,activation='relu'))
  model.add(Dense(64,activation='relu'))
  model.add(Dense(num_classes,activation='softmax'))
  return model


def create_rbf_model(input_shape,num_classes,units=64,gamma=1.0):
  model=Sequential()
  model.add(Flatten(input_shape=input_shape))
  model.add(RBFLayer(units,gamma))
  model.add(Dense(num_classes,activation='softmax'))
  return model

In [3]:
#loading MNIST dataset and normalizing the images:
from tensorflow.keras.datasets import mnist
(train_images,train_labels),(test_images,test_labels)= mnist.load_data()

train_images=train_images/255.0
test_images=test_images/255.0

11490434/11490434 [==============================] - 2s 0us/step


In [4]:
#creating models and compiling them:
num_classes=10
input_shape=(28,28)

dense_model=create_dense_model(input_shape,num_classes)
rbf_model=create_rbf_model(input_shape,num_classes)

dense_model.compile(optimizer='adam',
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                    metrics=['accuracy'])

rbf_model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

In [5]:
#training the model:
import time 
start_dense_time=time.time()
dense_history=dense_model.fit(train_images,train_labels,epochs=5,
                              validation_data=(test_images,test_labels))
end_dense_time=time.time()
start_rbf_time=time.time()
rbf_history=rbf_model.fit(train_images, train_labels, epochs=5,
                          validation_data=(test_images,test_labels))
end_rbf_time=time.time()

Epoch 1/5
1875/1875 [==============================] - 12s 4ms/step - loss: 0.2811 - accuracy: 0.9178 - val_loss: 0.1540 - val_accuracy: 0.9539
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1253 - accuracy: 0.9622 - val_loss: 0.1145 - val_accuracy: 0.9640
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0915 - accuracy: 0.9713 - val_loss: 0.1134 - val_accuracy: 0.9642
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0719 - accuracy: 0.9774 - val_loss: 0.0843 - val_accuracy: 0.9731
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0593 - accuracy: 0.9810 - val_loss: 0.0890 - val_accuracy: 0.9744
Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 2.3015 - accuracy: 0.1119 - val_loss: 2.3010 - val_accuracy: 0.1135
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3013 - accuracy: 0.1124 - val_loss: 2.3011 - val_accuracy: 0.113

How many parameters each of those networks have?

In [6]:
#this can be found using the count_params() function of the model after it has been built.
print("Dense model parameters: ", dense_model.count_params())
print("RBF model parameters: ", rbf_model.count_params())

Dense model parameters:  55050
RBF model parameters:  50890


Which network trains faster / easier?

In [7]:
print("Dense Model Training Time: ", end_dense_time - start_dense_time)
print("RBF Model Training Time: ", end_rbf_time - start_rbf_time)

Dense Model Training Time:  42.06096053123474
RBF Model Training Time:  28.53823161125183


given that Dense Layer involves straightforward matrix multiplications, while the RBF layer requires additional computation fo rthe gaussian function (including exponentials), so we cna say network with dense layer woul dgenerally train faster than RBL layer netowrk of similar size. Also, as you can observe in above two code snippets, parameters in rbf model is less so it is taking less time and trained faster than dense layer netowrk.